In [5]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding='latin-1')
df.columns = ['label', 'text']
df = df.sample(50)

In [6]:
from transformers import pipeline

classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [7]:
def classify_with_llm(text):
    prompt = f"Classify this email as 'spam' or 'ham': {text}"
    result = classifier(prompt)[0]
    return result['label']

df['predicted_label'] = df['text'].apply(classify_with_llm)


In [8]:
from sklearn.metrics import classification_report

print(classification_report(df['label'], df['predicted_label']))


                                                                                                                                                                                                                                                                                                     precision    recall  f1-score   support

                                                                                                                                                                                                                                        After completed degree. There is no use in joining finance.       0.00      0.00      0.00       1.0
                                                                                                                                   Aiyo... U always c our ex one... I dunno abt mei, she haven reply... First time u reply so fast... Y so lucky not workin huh, got bao by ur sugardad ah...gee..        0.00      0.00      0.00       1.0


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [9]:
print(df[['text', 'label', 'predicted_label']].head(5))


      text                                              label predicted_label
1264   ham                         I will see in half an hour        NEGATIVE
3114   ham                 I wait 4 Ã_ inside da car park...        NEGATIVE
1662  spam  Hi if ur lookin 4 saucy daytime fun wiv busty ...        NEGATIVE
377    ham  Well there's not a lot of things happening in ...        NEGATIVE
4808   ham  Don't worry though, I understand how important...        NEGATIVE


In [11]:
!pip install streamlit transformers pyngrok --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [12]:
%%writefile app.py
import streamlit as st
from transformers import pipeline

st.title("🤖 Chatbot GPT-2")

chatbot = pipeline("text-generation", model="gpt2")

prompt = st.text_input("Vous :", "")

if prompt:
    response = chatbot(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']
    st.text_area("Réponse :", value=response, height=200)


Writing app.py


In [14]:
from pyngrok import ngrok
import time
import subprocess

# Fermer les connexions ngrok existantes si besoin
ngrok.kill()

# Démarrer Streamlit
streamlit_proc = subprocess.Popen(["streamlit", "run", "app.py"])

# Attendre quelques secondes que Streamlit démarre
time.sleep(5)

# Créer le tunnel ngrok
public_url = ngrok.connect(8501)
print("🌐 Cliquez ici pour accéder à votre app :", public_url)


ERROR:pyngrok.process.ngrok:t=2025-07-08T14:43:41+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-08T14:43:41+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.